## Goal

We’ll use a simple GAN to generate frog images from the CIFAR dataset


### The Dataset

CIFAR has 10 image classes with 5,000 training images of dimension 32X32 in RGB.


### Image Generation Program

1. A generator network $G$ maps images of shape `(latent_dim,)` to `(32,32,3)` , so the generator map is $G: \mathbb{R}\rightarrow \mathbb{R}^3$
2. A discriminator network $D$ maps the images from `(32,32,3)`, to a binary score $`[0,1]` predicting whether the image is real or fake
3.  A GAN network chains $D$ and $G$ so $GAN = D(G(x))$, so the GAN map is $G: \mathbb{R} \rightarrow \mathbb{R}^3 \rightarrow [0,1]$, from `(latent_dim,)` to `(32,32,3)` to `[0,1]`
4. $D$ is trained with a collection of fake images (from $G$) and real images from the training set
5. To train $G$, you adjust the weights in $G$ with regard to the loss of the $GAN$ model, so $G$ weights are updated in the directions that makes $D$ more likely classify fake images as real




In [22]:
# import relevant libs

import tensorflow
import keras
#progress bar for keras

keras.__version__

'2.2.0'

In [23]:
# check if machine has GPU

from tensorflow.python.client import device_lib

device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 12682005189347036066, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 11270533940
 locality {
   bus_id: 1
   links {
     link {
       device_id: 1
       type: "StreamExecutor"
       strength: 1
     }
   }
 }
 incarnation: 4471602824184595817
 physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7", name: "/device:GPU:1"
 device_type: "GPU"
 memory_limit: 11285538407
 locality {
   bus_id: 1
   links {
     link {
       type: "StreamExecutor"
       strength: 1
     }
   }
 }
 incarnation: 14077058396035838503
 physical_device_desc: "device: 1, name: Tesla K80, pci bus id: 0000:00:05.0, compute capability: 3.7"]

In [24]:
# Generator 


from keras import layers
import numpy as np

#latent vector
latent_dim = 32
height = 32
width = 32
channels = 3

generator_input = keras.Input(shape=(latent_dim,))

#Transforms the generator input (latent_dim,) to a 16X16 128 channel feature map
x = layers.Dense(128 * 16 * 16)(generator_input)
x = layers.LeakyReLU()(x)
x = layers.Reshape((16, 16, 128))(x)
x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)

#Transforms to 32 X 32
x = layers.Conv2DTranspose(256, 4, strides=2, padding='same')(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)

# These 3 lines produce a 32X32 1 channel image, which is the shape of CIFAR10 images
x = layers.Conv2D(channels, 7, activation='tanh', padding='same')(x)\

# Instantiates generator model which maps (latent_dim,) to (32,32,3)
generator = keras.models.Model(generator_input, x)
generator.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 32)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 32768)             1081344   
_________________________________________________________________
leaky_re_lu_10 (LeakyReLU)   (None, 32768)             0         
_________________________________________________________________
reshape_2 (Reshape)          (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 16, 16, 256)       819456    
_________________________________________________________________
leaky_re_lu_11 (LeakyReLU)   (None, 16, 16, 256)       0         
_________________________________________________________________
conv2d_transpose_2 (Conv2DTr (None, 32, 32, 256)       1048832   
__________

In [25]:
# Discriminator


discriminator_input = layers.Input(shape=(height, width, channels))
x = layers.Conv2D(128, 3)(discriminator_input)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Flatten()(x)
    
# Trick 1: Dropout layer
x = layers.Dropout(0.4)(x)
# Classification step
x = layers.Dense(1, activation='sigmoid')(x)
# instantiates discriminator which turns (32,32,3) to [0,1]
discriminator = keras.models.Model(discriminator_input, x)
discriminator.summary()
discriminator_optimizer = keras.optimizers.RMSprop(
        lr=0.0008,
        clipvalue=1.0,
        decay=1e-8)
discriminator.compile(optimizer=discriminator_optimizer,loss='binary_crossentropy')


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 30, 30, 128)       3584      
_________________________________________________________________
leaky_re_lu_15 (LeakyReLU)   (None, 30, 30, 128)       0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 14, 14, 128)       262272    
_________________________________________________________________
leaky_re_lu_16 (LeakyReLU)   (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 6, 6, 128)         262272    
_________________________________________________________________
leaky_re_lu_17 (LeakyReLU)   (None, 6, 6, 128)         0         
__________

In [26]:
# Adversarial Network

# This sets discriminator weights as untrainable
discriminator.trainable = False

gan_input = keras.Input(shape=(latent_dim,)) 

gan_output = discriminator(generator(gan_input)) 

gan = keras.models.Model(gan_input, gan_output)

gan_optimizer = keras.optimizers.RMSprop(lr=0.0004, clipvalue=1.0, decay=1e-8) 

gan.compile(optimizer=gan_optimizer, loss='binary_crossentropy')

## GAN Algorithm Steps

### Generate Data


1. Create random latent space data (random noise in 1D vector) - shape `(latent_dim,)`
2. Generate images from latent space date - shape `(32,32,3)`

### Mix Fake + Real Data


3. Mix real + fake images in one dataset (with targets real as 1 and fake as 0)

### Train Discriminator


4. Train discriminator with these images

### Train Generator to Fool Discriminator


5. Create new latent space random vectors
6. Label them as real 
7. Train GAN with new random vectors (this step teaches G to fool D)


In [27]:
# run the GAN algo

import os
from keras.preprocessing import image
# load CIFAR data
(x_train, y_train), (_, _) = keras.datasets.cifar10.load_data()

In [28]:
# select frogs
x_train = x_train[y_train.flatten() == 6]

# normalize data
x_train = x_train.reshape(
    (x_train.shape[0],) +
    (height, width, channels)).astype('float32') / 255.



In [29]:
# specify where to save data, iteration, and batch size

iterations = 10000
batch_size = 20
save_dir = 'data'

In [30]:
start = 0

for step in range(iterations):
    
    #create random latent space vectors
    random_latent_vectors = np.random.normal(size=(batch_size,
                                        latent_dim))
    
    #convert vectors to images
    generated_images = generator.predict(random_latent_vectors)
    
    #combine fake + real images
    stop = start + batch_size
    real_images = x_train[start: stop]
    combined_images = np.concatenate([generated_images, real_images])
    
    #generate labels for fake / real images (true labels, no fooling here)
    labels = np.concatenate([np.ones((batch_size, 1)),
                         np.zeros((batch_size, 1))])
    
    #Trick: add random noise to labels
    labels += 0.05 * np.random.random(labels.shape)
    
    #Train discriminator
    d_loss = discriminator.train_on_batch(combined_images, labels)
    
    #create another batch of random latent space vectors
    random_latent_vectors = np.random.normal(size=(batch_size,latent_dim))
    
    #label these new vectors as real (this is a lie! this will be used to train G to fool D here)
    misleading_targets = np.zeros((batch_size, 1))
    
    #trains G, recall that D's weights are frozen
    a_loss = gan.train_on_batch(random_latent_vectors,
                            misleading_targets)
    
    #advance start position by batch_size
    start += batch_size
    if start > len(x_train) - batch_size:
        start = 0
        
    #if step % 5 == 0:
    #    print('step',step)
    #save weights every 100 steps and plot image
    if step % 100 == 0:
        gan.save_weights('gan_gpu.h5')
        
        #print loss metrics
        print('step',step)
        print('discriminator loss:', d_loss)
        print('adversarial loss:', a_loss)
        
        #save fake image
        img = image.array_to_img(generated_images[0] * 255., scale=False)
        img.save(os.path.join('/home/laskin_misha/GANs/'+save_dir,'gpu_generated_frog' + str(step) + '.png'))
        
        #save real image
        img = image.array_to_img(real_images[0] * 255., scale=False)
        img.save(os.path.join('/home/laskin_misha/GANs/'+save_dir,'gpu_real_frog' + str(step) + '.png'))

/home/laskin_misha/.local/lib/python3.5/site-packages/keras/engine/training.py:478: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


step 0
discriminator loss: 0.7083238
adversarial loss: 0.6945063
step 100
discriminator loss: 0.7151143
adversarial loss: 0.7071212
step 200
discriminator loss: 0.7001738
adversarial loss: 0.80984926
step 300
discriminator loss: 0.69444656
adversarial loss: 0.77848965
step 400
discriminator loss: 0.69765127
adversarial loss: 0.735351
step 500
discriminator loss: 0.6959867
adversarial loss: 0.7793239
step 600
discriminator loss: 0.69855773
adversarial loss: 0.7537593
step 700
discriminator loss: 0.69748765
adversarial loss: 0.74746025
step 800
discriminator loss: 0.7836197
adversarial loss: 0.78498536
step 900
discriminator loss: 0.68427706
adversarial loss: 0.7280313
step 1000
discriminator loss: 0.6965578
adversarial loss: 0.76312315
step 1100
discriminator loss: 0.6948187
adversarial loss: 0.74227273
step 1200
discriminator loss: 0.68762124
adversarial loss: 0.98710805
step 1300
discriminator loss: 0.6877942
adversarial loss: 0.75625324
step 1400
discriminator loss: 0.73740304
advers